In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import pickle
import sqlite3


In [4]:
pip install pandas


  Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl (11.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
df = pd.read_csv('Online_Courses.csv')

In [7]:
df.head()
df.shape


(8092, 45)

In [8]:
columns_list = df.columns.tolist()

In [9]:
print(columns_list)

['Unnamed: 0', 'Title', 'URL', 'Short Intro', 'Category', 'Sub-Category', 'Course Type', 'Language', 'Subtitle Languages', 'Skills', 'Instructors', 'Rating', 'Number of viewers', 'Duration', 'Site', 'Program Type', 'Courses', 'Level', 'Number of Reviews', 'Unique Projects', 'Prequisites', 'What you learn', 'Related Programs', 'Monthly access', '6-Month access', '4-Month access', '3-Month access', '5-Month access', '2-Month access', 'School', 'Topics related to CRM', 'ExpertTracks', 'FAQs', 'Course Title', 'Course URL', 'Course Short Intro', 'Weekly study', 'Premium course', "What's include", 'Rank', 'Created by', 'Program', 'Number of ratings', 'Price', 'COURSE CATEGORIES']


In [10]:
columns_to_keep = ['Unnamed: 0', 'Title', 'URL', 'Short Intro', 'Category', 'Sub-Category', 'Course Type', 'Language', 'Subtitle Languages', 'Skills', 'Instructors', 'Rating', 'Number of viewers', 'Duration', 'Site']
df = df.loc[:, columns_to_keep]
df.head()

,Unnamed: 0,Title,URL,Short Intro,Category,Sub-Category,Course Type,Language,Subtitle Languages,Skills,Instructors,Rating,Number of viewers,Duration,Site
0,0,Machine Learning Specialization,https://www.coursera.org/specializations/machi...,#BreakIntoAI with Machine Learning Specializat...,Data Science,Machine Learning,Specialization,English,Subtitles: English,"Decision Trees, Artificial Neural Network, Log...","Andrew Ng ,Eddy Shyu ,Aarti Bagul ,Geoff Ladwig ,",4.9stars,"10,438",Approximately 3 months to complete,Coursera
1,1,Introduction to Data Science Specialization,https://www.coursera.org/specializations/intro...,Launch your career in data science. Gain found...,Data Science,Data Analysis,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Relational Database Management S...","Rav Ahuja ,Alex Aklson ,Aije Egwaikhide ,Svetl...",4.7stars,"11,927",Approximately 5 months to complete,Coursera
2,2,Data Science Fundamentals with Python and SQL ...,https://www.coursera.org/specializations/data-...,Build the Foundation for your Data Science car...,Data Science,Data Analysis,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Github, Python Programming, Jupy...","Aije Egwaikhide ,Svetlana Levitan ,Romeo Kienz...",4.6stars,"2,295",Approximately 7 months to complete,Coursera
3,3,Key Technologies for Business Specialization,https://www.coursera.org/specializations/key-t...,Get Ahead with Key Business Technologies. Gain...,Business,Business Essentials,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Artificial Intelligence (AI), Bu...","Rav Ahuja ,Alex Aklson ,",4.7stars,"1,232",Approximately 3 months to complete,Coursera
4,4,Deep Learning Specialization,https://www.coursera.org/specializations/deep-...,Become a Machine Learning expert. Master the f...,Data Science,Machine Learning,Specialization,English,"Subtitles: English, Chinese (Traditional), Ara...","Artificial Neural Network, Convolutional Neura...","Andrew Ng ,Kian Katanforoosh ,Younes Bensouda ...",4.9stars,"129,253",Approximately 5 months to complete,Coursera


In [11]:
df = df.rename(columns={'Unnamed: 0': 'course_id'})
df['course_id'] += 101
df.head()


,course_id,Title,URL,Short Intro,Category,Sub-Category,Course Type,Language,Subtitle Languages,Skills,Instructors,Rating,Number of viewers,Duration,Site
0,101,Machine Learning Specialization,https://www.coursera.org/specializations/machi...,#BreakIntoAI with Machine Learning Specializat...,Data Science,Machine Learning,Specialization,English,Subtitles: English,"Decision Trees, Artificial Neural Network, Log...","Andrew Ng ,Eddy Shyu ,Aarti Bagul ,Geoff Ladwig ,",4.9stars,"10,438",Approximately 3 months to complete,Coursera
1,102,Introduction to Data Science Specialization,https://www.coursera.org/specializations/intro...,Launch your career in data science. Gain found...,Data Science,Data Analysis,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Relational Database Management S...","Rav Ahuja ,Alex Aklson ,Aije Egwaikhide ,Svetl...",4.7stars,"11,927",Approximately 5 months to complete,Coursera
2,103,Data Science Fundamentals with Python and SQL ...,https://www.coursera.org/specializations/data-...,Build the Foundation for your Data Science car...,Data Science,Data Analysis,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Github, Python Programming, Jupy...","Aije Egwaikhide ,Svetlana Levitan ,Romeo Kienz...",4.6stars,"2,295",Approximately 7 months to complete,Coursera
3,104,Key Technologies for Business Specialization,https://www.coursera.org/specializations/key-t...,Get Ahead with Key Business Technologies. Gain...,Business,Business Essentials,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Artificial Intelligence (AI), Bu...","Rav Ahuja ,Alex Aklson ,",4.7stars,"1,232",Approximately 3 months to complete,Coursera
4,105,Deep Learning Specialization,https://www.coursera.org/specializations/deep-...,Become a Machine Learning expert. Master the f...,Data Science,Machine Learning,Specialization,English,"Subtitles: English, Chinese (Traditional), Ara...","Artificial Neural Network, Convolutional Neura...","Andrew Ng ,Kian Katanforoosh ,Younes Bensouda ...",4.9stars,"129,253",Approximately 5 months to complete,Coursera


In [12]:
df['course_features'] = df['Title'] + ' ' + df['Category'] + ' ' + df['Sub-Category'] + ' ' + df['Skills']
df.head()

,course_id,Title,URL,Short Intro,Category,Sub-Category,Course Type,Language,Subtitle Languages,Skills,Instructors,Rating,Number of viewers,Duration,Site,course_features
0,101,Machine Learning Specialization,https://www.coursera.org/specializations/machi...,#BreakIntoAI with Machine Learning Specializat...,Data Science,Machine Learning,Specialization,English,Subtitles: English,"Decision Trees, Artificial Neural Network, Log...","Andrew Ng ,Eddy Shyu ,Aarti Bagul ,Geoff Ladwig ,",4.9stars,"10,438",Approximately 3 months to complete,Coursera,Machine Learning Specialization Data Science M...
1,102,Introduction to Data Science Specialization,https://www.coursera.org/specializations/intro...,Launch your career in data science. Gain found...,Data Science,Data Analysis,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Relational Database Management S...","Rav Ahuja ,Alex Aklson ,Aije Egwaikhide ,Svetl...",4.7stars,"11,927",Approximately 5 months to complete,Coursera,Introduction to Data Science Specialization Da...
2,103,Data Science Fundamentals with Python and SQL ...,https://www.coursera.org/specializations/data-...,Build the Foundation for your Data Science car...,Data Science,Data Analysis,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Github, Python Programming, Jupy...","Aije Egwaikhide ,Svetlana Levitan ,Romeo Kienz...",4.6stars,"2,295",Approximately 7 months to complete,Coursera,Data Science Fundamentals with Python and SQL ...
3,104,Key Technologies for Business Specialization,https://www.coursera.org/specializations/key-t...,Get Ahead with Key Business Technologies. Gain...,Business,Business Essentials,Specialization,English,"Subtitles: English, Arabic, French, Portuguese...","Data Science, Artificial Intelligence (AI), Bu...","Rav Ahuja ,Alex Aklson ,",4.7stars,"1,232",Approximately 3 months to complete,Coursera,Key Technologies for Business Specialization B...
4,105,Deep Learning Specialization,https://www.coursera.org/specializations/deep-...,Become a Machine Learning expert. Master the f...,Data Science,Machine Learning,Specialization,English,"Subtitles: English, Chinese (Traditional), Ara...","Artificial Neural Network, Convolutional Neura...","Andrew Ng ,Kian Katanforoosh ,Younes Bensouda ...",4.9stars,"129,253",Approximately 5 months to complete,Coursera,Deep Learning Specialization Data Science Mach...


In [13]:
tfidf = TfidfVectorizer()
features_matrix = tfidf.fit_transform(df['course_features'].fillna('').astype(str))

In [14]:
cosine_sim = linear_kernel(features_matrix, features_matrix)

In [15]:
def get_recommendations(user_preferences, n=5):
    recommended_courses = set()
    # Convert user preferences into TF-IDF vectors
    user_pref_matrix = tfidf.transform(user_preferences)
    # Compute cosine similarity between user preferences and course features
    sim_scores = linear_kernel(user_pref_matrix, features_matrix)
    # Get indices of courses with highest similarity scores
    top_indices = sim_scores.argsort()[0][-n-1:-1][::-1]  # Exclude the user preference itself
    # Get titles of recommended courses
    recommended_courses.update(df.iloc[top_indices]['Title'])
    
    return recommended_courses


In [16]:
user_preferences = ['Machine Learning']
recommendations = get_recommendations(user_preferences)
print(f"Recommendations based on user preferences: {recommendations}")

Recommendations based on user preferences: {'Machine Learning Foundations: A Case Study Approach', 'Practical Machine Learning', 'Managing Machine Learning Projects', 'Machine Learning With Big Data'}


In [17]:


# Save trained components into a pickle file
with open('recommendation_model1.pkl', 'wb') as f:
    pickle.dump((tfidf, features_matrix), f)

In [18]:
# Assuming you have a DataFrame df containing your dataset

# Create a connection to SQLite database
conn = sqlite3.connect(r'C:\Users\PAVAN\OneDrive\Desktop\Course_Rec_Project\api\demo.db')

# Save the DataFrame to the database
df.to_sql('Courses_data', conn, if_exists='replace', index=False)

# Close the connection
conn.close()